In [ ]:
# Python Packages for this code:

# python3 -m pip install --upgrade numpy opencv-python tensorflow imageio pillow jupyter

In [30]:
import pathlib
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.resnet50 import preprocess_input

cascade_path = pathlib.Path(cv2.__file__).parent.absolute() / "data/haarcascade_frontalface_default.xml"

In [31]:
clf = cv2.CascadeClassifier(str(cascade_path))

In [32]:
# Button Region

button_location = (50,50,300,120)
quitbuttonpressed = False

In [33]:
# Mouse Click Detection Function and change boolean variable

def mouse_click(event,x,y,flags,param):
    global quitbuttonpressed
    if event == cv2.EVENT_LBUTTONDOWN:
        x1,y1,x2,y2 = button_location
        if x1 <= x <= x2 and y1 <= y <= y2:
            quitbuttonpressed = True

In [34]:
cv2.namedWindow("Faces")
cv2.setMouseCallback("Faces", mouse_click)

In [35]:
# Camera Load

camera = cv2.VideoCapture(1) # Either 0 or 1, dependent on whther a phone is connected to the computer

In [36]:
# Load the dog images

dogimages = {
    "tongue_out": cv2.imread("dogimages/tongue_out.jpg"),
    "eyesclosed": cv2.imread("dogimages/eyesclosed.jpg"),
    "perplexed": cv2.imread("dogimages/perplexed.JPG"),
    "scream": cv2.imread("dogimages/scream.jpg"),
    "smile": cv2.imread("dogimages/smile.jpg")            
}

for doggy in dogimages:
    dogimages[doggy] = cv2.resize(dogimages[doggy], (480,480))

In [ ]:
# Change this to either v1,2 or 3 depending on which model to test.

model = load_model("CustomNetworkv1.keras")


classes = ["eyesclosed",
           "scream",
           "tongue_out"]

# Redundant, however the for flow of code it works fine.
model_to_dog = {
    'eyesclosed': 'eyesclosed',
    'scream': 'scream',
    'tongue_out': 'tongue_out'
}

In [38]:
# Resize detected faces to 224x224, preprocess the image data

def prepare_face(face_img):
    resized = cv2.resize(face_img, (224,224))
    img_array = np.expand_dims(resized,axis=0)
    img_array = preprocess_input(img_array)
    return img_array

In [39]:
# Function for centred text

def _put_centered_text(img, text, color=(255,255,255), scale=1.1, thickness=2):
    font = cv2.FONT_HERSHEY_SIMPLEX
    (tw, th), baseline = cv2.getTextSize(text, font, scale, thickness)
    x = (img.shape[1] - tw) // 2
    y = (img.shape[0] + th) // 2
    cv2.putText(img, text, (x, y), font, scale, color, thickness, cv2.LINE_AA)

# Fade out function

def _fade_out(img, steps=30, delay=30):
    for alpha in np.linspace(1.0, 0.0, steps):
        frame = cv2.convertScaleAbs(img, alpha=alpha, beta=0)
        cv2.imshow("Faces", frame)
        if cv2.waitKey(delay) & 0xFF == ord('q'):
            break

# Function to play intro gif

def _play_gif(gif_path, duration_sec=2.0, size=(480,960), window="Faces"):
    import time
    try:
        import imageio.v2 as imageio
    except ImportError:
        return None
    last_bgr = None
    try:
        reader = imageio.get_reader(gif_path)
    except Exception:
        return None
    t_end = time.time() + duration_sec
    try:
        while time.time() < t_end:
            for frame in reader:
                if time.time() >= t_end:
                    break
                # Convert RGB/RGBA -> BGR
                if frame.ndim == 3 and frame.shape[2] == 4:
                    bgr = cv2.cvtColor(frame, cv2.COLOR_RGBA2BGR)
                else:
                    bgr = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
                bgr = cv2.resize(bgr, size)
                last_bgr = bgr
                cv2.imshow(window, bgr)
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
    finally:
        try:
            reader.close()
        except Exception:
            pass
    return last_bgr

# Animated gif, fade to black
base = np.zeros((480, 960, 3), dtype=np.uint8)
last_frame = _play_gif("intrologoanimated.gif", duration_sec=2.0, size=(960,480), window="Faces")
# if the gif isn't available, use a fallback
if last_frame is None:
    fallback = base.copy()
    _put_centered_text(fallback, "what dog am i", color=(255,255,255), scale=1.2, thickness=2)
    cv2.imshow("Faces", fallback)
    cv2.waitKey(2000)
    last_frame = fallback
_fade_out(last_frame)

# Confirmation that camera is working
camera_ok = camera.isOpened()
status = base.copy()
if camera_ok:
    _put_centered_text(status, "Camera working!", color=(0,255,0), scale=1.0, thickness=2)
else:
    _put_centered_text(status, "Camera not working!", color=(0,0,255), scale=1.0, thickness=2)

cv2.imshow("Faces", status)
cv2.waitKey(1500)
_fade_out(status)

if not camera_ok:
    raise RuntimeError("Camera isn't working!")


In [40]:
while True:
    ret,frame = camera.read()
    if not ret or frame is None:
        print("Frame not captured.")
        continue
    

    height = 480

    h,w,c = frame.shape
    min_dim = min(h,w)
    start_x = w//2 - min_dim//2
    start_y = h//2 - min_dim//2

    square_frame = frame[start_y:start_y+min_dim, start_x:start_x+min_dim]
    display_frame = cv2.resize(square_frame, (480,480))

    gray = cv2.cvtColor(display_frame, cv2.COLOR_BGR2GRAY)
    faces = clf.detectMultiScale(
        gray,
        scaleFactor = 1.05,
        minNeighbors=5,
        minSize=(10,10),
        flags=cv2.CASCADE_SCALE_IMAGE
    )

    # If a face is detected, classify it and display the dog images based off the classifier
    
    if len(faces) > 0:
        faces_sorted = sorted(faces, key=lambda b: b[2]*b[3], reverse=True)
        x,y,wf,hf = faces_sorted[0]

        cv2.rectangle(display_frame ,(x,y),(x+wf, y+hf),(255,255,0),2)

        face_roi = display_frame[y:y+hf, x:x+wf]
        input_array = prepare_face(face_roi)
        preds = model.predict(input_array, verbose = 0)
        pred_class = classes[np.argmax(preds)]
        current_dog = dogimages[model_to_dog[pred_class]]
    else:
        current_dog = dogimages["perplexed"]

    # Draw quit button

    x1, y1, x2, y2 = button_location
    cv2.rectangle(display_frame, (x1, y1), (x2, y2), (0,0,255),-1)
    cv2.putText(display_frame, "Press to Quit", (x1 + 20, y1  + 45),
                 cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255,255,255),2)
    
    combined = np.hstack((display_frame,current_dog))
    cv2.imshow("Faces", combined)

    if quitbuttonpressed or cv2.waitKey(1) & 0xFF == ord('q'):
        break

camera.release
cv2.destroyAllWindows()